In [158]:
import pandas as pd
import numpy as np

import sys
sys.path.insert(1, 'dnn/')

# del bnn
import bcell_nn as bnn
import bcell_plot
import bcell_preprocess as bpreprocess
# import bcell_driver

# System arguments
import argparse
# data management
import numpy as np
import pandas as pd
# visualization
import matplotlib.pyplot as plt
import seaborn as sns
# flow cytometry libraries
import cytoflow as flow
# user defined functions

from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline

In [160]:
# concat wishbone data
branches = pd.concat([pd.read_csv('../data/Wishbone_branches/branches_bcr_signalling.csv', index_col=0), pd.read_csv('../data/Wishbone_branches/branches_basal_signalling.csv', index_col=0)])
trajectories = pd.concat([pd.read_csv('../data/Wishbone_trajectories/trajectory_bcr_signalling.csv', index_col=0), pd.read_csv('../data/Wishbone_trajectories/trajectory_basal_signalling.csv', index_col=0)])

branches.columns = ['branch']
trajectories.columns = ['trajectory']

branches = branches[~branches.index.duplicated(keep='first')]
trajectories = trajectories[~trajectories.index.duplicated(keep='first')]

print(len(branches), len(trajectories))

29238 29238


In [188]:
df1 = pd.read_csv('../data/Bcell_signalling_proteins/matrix_basal_signalling_markers_#2.csv', index_col=0)
df2 = pd.read_csv('../data/Bcell_signalling_proteins/matrix_bcr_signalling_markers_#2.csv', index_col=0)
df1['bcr'] = 0
df2['bcr'] = 1

signal_df = pd.concat([df1, df2])
signal_df.head()

print(signal_df.shape)

(29702, 23)


In [189]:
signal_df = signal_df.merge(branches, left_index=True, right_index=True, how='inner')
signal_df = signal_df.merge(trajectories, left_index=True, right_index=True, how='inner')
signal_df

,110_114-CD3,111-CD3,112-CD3,114-CD3,141-pPLCgamma2,150-pSTAT5,151-pERK1/2,152-Ki67,153-pMAPKAPK2,154-pSHP2,...,168-pH3,169-pP38,171-pBtk/Itk,172-pS6,174-pSrcFK,175-pCrkL,176-pCREB,bcr,branch,trajectory
8,5.171318,-0.450817,-0.608567,6.029278,9.331525,-0.845049,14.676166,1.818566,2.874730,17.832912,...,8.435942,76.529343,4.326055,0.655243,2.677969,-1.446438,2.424710,0,2,0.264602
12,-13.813409,0.754631,-1.202560,-11.105885,84.821815,7.536618,38.950451,1.903706,13.491470,26.370272,...,64.488258,115.278679,17.951509,29.039434,62.710323,6.291503,31.427765,1,1,0.057659
30,0.352439,0.341111,-1.615738,-0.122326,1.303879,2.899095,34.662552,-0.275870,15.783648,17.620398,...,5.382995,101.783089,6.834910,28.550144,34.058968,4.541006,18.538198,1,1,0.058571
45,-2.647034,-0.061374,2.036048,0.190818,7.546767,10.631545,16.491894,3.028020,6.266270,4.856538,...,22.576366,151.329102,1.419633,33.557350,11.859700,8.606210,12.979078,1,1,0.058252
73,0.013113,-0.688039,0.232603,-0.838379,18.820129,8.742398,22.354654,-3.074918,1.138519,1.086791,...,7.996578,63.526962,29.082439,29.092857,21.322346,4.467139,10.174027,1,1,0.058832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471739,-2.799587,-0.607517,-0.835767,-0.980455,322.782074,16.849302,95.306534,0.570192,29.137192,51.822701,...,35.923096,39.349442,6.410147,93.473450,62.584103,15.369135,11.190707,1,1,0.055541
471770,23.045057,14.629941,-0.410597,3.637822,27.408026,11.361174,99.938850,0.273641,34.619743,14.185965,...,55.876968,47.855972,34.769245,37.195259,63.399906,8.008888,33.619480,1,1,0.058429
471970,3.909767,-0.020133,-1.045324,5.840205,17.922468,11.660793,22.337528,-0.332774,6.761919,0.654727,...,40.555592,62.141502,6.441080,88.479965,40.739178,5.325276,50.402176,1,1,0.058701
472012,3.447302,-0.439497,3.060533,-0.869404,6.576580,6.294831,15.881731,-0.756070,11.646162,4.539165,...,21.692110,76.187645,11.464962,23.188345,62.133389,2.555749,19.671515,1,1,0.058686


In [190]:
# pop off last .1 as test data
np.random.seed(0)
signal_df_train = signal_df.sample(frac=0.8)

signal_test = signal_df[~signal_df.index.isin(signal_df_train.index)]

In [192]:

signal_df_train.to_csv('../data/signal_train.csv')
signal_test.to_csv('../data/signal_test.csv')

In [177]:
df1 = pd.read_csv( '../data/Bcell_surface_marker/matrix_basal_surface_markers_#2.csv', index_col=0)
df2 = pd.read_csv( '../data/Bcell_surface_marker/matrix_bcr_surface_markers_#2.csv', index_col=0)

df1['bcr'] = 0
df2['bcr'] = 1

surface_df = pd.concat([df1, df2])
surface_df.head()
print(surface_df.shape)

(29702, 14)


In [178]:
surface_df = surface_df.merge(branches, left_index=True, right_index=True, how='inner')
surface_df = surface_df.merge(trajectories, left_index=True, right_index=True, how='inner')
surface_df

,110-CD3,115-CD45,139-CD45RA,142-CD19,144-CD11b,145-CD4,146-CD8,147-CD20,148-CD34,158-CD33,160-CD123,167-CD38,170-CD90,bcr,branch,trajectory
8,-0.072176,219.463165,73.935654,36.863819,-1.054267,-0.147575,-0.102538,32.607048,-0.249687,0.067650,6.987092,37.832825,2.822956,0,2,0.264602
12,-2.307435,204.438171,77.867607,42.641132,-0.285443,-0.372202,-4.427719,17.261219,-0.488574,-3.711010,34.505669,19.478870,5.564790,1,1,0.057659
30,1.746720,284.282013,28.760956,38.499954,1.275578,-1.653929,-0.024098,39.808941,-0.463562,0.135652,50.319153,22.642670,-0.418175,1,1,0.058571
45,-4.815868,234.770752,62.574379,44.692940,3.540209,-0.526092,0.172865,40.247784,2.179591,5.230491,-0.804634,59.010201,-0.574145,1,1,0.058252
73,1.310114,108.836357,48.661461,34.506725,-1.216142,-0.464221,2.820165,37.411366,-0.315401,0.303836,22.056704,23.780251,4.541167,1,1,0.058832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471739,-0.383450,246.238373,50.226032,14.160948,-0.589054,1.851488,-0.115561,27.474073,0.068921,1.629810,3.316009,1396.733887,3.400850,1,1,0.055541
471770,5.128566,284.982178,94.956169,77.147202,7.554247,1.254417,2.702371,22.974083,4.606533,4.899467,1.558595,14.952886,3.802980,1,1,0.058429
471970,-0.838438,340.048737,31.034472,53.579376,-0.292885,1.439927,-0.062041,74.637009,7.424857,-0.148461,43.309162,68.566292,1.943407,1,1,0.058701
472012,1.715871,317.436554,28.813572,42.742043,-0.876804,1.381478,-0.605978,41.627674,5.863369,0.977718,17.094145,19.128323,0.636132,1,1,0.058686


In [182]:
# pop off last .1 as test data
np.random.seed(0)
surface_df_train = surface_df.sample(frac=0.8)

# to recover 
surface_test = surface_df[~surface_df.index.isin(surface_df_train.index)]

In [185]:
surface_df_train.to_csv('../data/surface_train.csv')
surface_test.to_csv('../data/surface_test.csv')

In [148]:
# train model
def get_model(input_shape):
    inputs = keras.Input(shape=(input_shape,))
    for i, x in enumerate([10, 8, 4]):
        if i == 0:
            outputs = layers.Dense(x, activation='relu')(inputs)
        else:
            outputs = layers.Dense(x, activation='relu')(outputs)
    outputs = layers.Dropout(0.1, name='dropout')(outputs)
    outputs = layers.Dense(1, activation='sigmoid')(outputs)

    model1 = keras.Model(inputs, outputs)
    
    
    return model1

# signal df

In [149]:
y = signal_df_train.pop('bcr')

In [194]:


model = get_model(signal_df_train.shape[1])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["acc"])

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
history = model.fit(signal_df_train, y,
                    batch_size = 64,
                    validation_split=0.1,
                    epochs=100
                   )

Epoch 1/100
335/335 [==============================] - 1s 3ms/step - loss: 3.2299 - acc: 0.5220 - val_loss: 0.6922 - val_acc: 0.5461
Epoch 2/100
335/335 [==============================] - 1s 3ms/step - loss: 0.7040 - acc: 0.5297 - val_loss: 0.6886 - val_acc: 0.5431
Epoch 3/100
335/335 [==============================] - 1s 3ms/step - loss: 0.6969 - acc: 0.5303 - val_loss: 0.6884 - val_acc: 0.5423
Epoch 4/100
335/335 [==============================] - 1s 3ms/step - loss: 0.6938 - acc: 0.5284 - val_loss: 0.6896 - val_acc: 0.5372
Epoch 5/100
335/335 [==============================] - 1s 3ms/step - loss: 0.6927 - acc: 0.5307 - val_loss: 0.6895 - val_acc: 0.5435
Epoch 6/100
335/335 [==============================] - 1s 3ms/step - loss: 0.6879 - acc: 0.5460 - val_loss: 0.6949 - val_acc: 0.5953
Epoch 7/100
335/335 [==============================] - 1s 3ms/step - loss: 0.6715 - acc: 0.5921 - val_loss: 0.6363 - val_acc: 0.6479
Epoch 8/100
335/335 [==============================] - 1s 3ms/step - 

335/335 [==============================] - 1s 3ms/step - loss: 0.0687 - acc: 0.9846 - val_loss: 0.0315 - val_acc: 0.9950
Epoch 63/100
335/335 [==============================] - 1s 3ms/step - loss: 0.0545 - acc: 0.9886 - val_loss: 0.0967 - val_acc: 0.9781
Epoch 64/100
335/335 [==============================] - 1s 3ms/step - loss: 0.0842 - acc: 0.9803 - val_loss: 0.0421 - val_acc: 0.9907
Epoch 65/100
335/335 [==============================] - 1s 3ms/step - loss: 0.0597 - acc: 0.9864 - val_loss: 0.0268 - val_acc: 0.9958
Epoch 66/100
335/335 [==============================] - 1s 3ms/step - loss: 0.0436 - acc: 0.9899 - val_loss: 0.0277 - val_acc: 0.9966
Epoch 67/100
335/335 [==============================] - 1s 3ms/step - loss: 0.0510 - acc: 0.9891 - val_loss: 0.0226 - val_acc: 0.9971
Epoch 68/100
335/335 [==============================] - 1s 3ms/step - loss: 0.0479 - acc: 0.9904 - val_loss: 0.0217 - val_acc: 0.9971
Epoch 69/100
335/335 [==============================] - 1s 3ms/step - loss:

In [195]:
bnn.save_model(model, '../models/wishbone_signal', history=history)

INFO:tensorflow:Assets written to: ../models/wishbone_signal2/assets


'../models/wishbone_signal2'

# surface df

In [197]:
y = surface_df_train.pop('bcr')

In [199]:
model = get_model(surface_df_train.shape[1])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["acc"])

In [200]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
history = model.fit(surface_df_train, y,
                    batch_size = 64,
                    validation_split=0.1,
                    epochs=100
                   )

Epoch 1/100
268/268 [==============================] - 1s 3ms/step - loss: 1.1170 - acc: 0.5044 - val_loss: 0.6925 - val_acc: 0.5334
Epoch 2/100
268/268 [==============================] - 1s 3ms/step - loss: 0.7038 - acc: 0.5221 - val_loss: 0.6909 - val_acc: 0.5339
Epoch 3/100
268/268 [==============================] - 1s 3ms/step - loss: 0.6977 - acc: 0.5221 - val_loss: 0.6910 - val_acc: 0.5334
Epoch 4/100
268/268 [==============================] - 1s 3ms/step - loss: 0.6941 - acc: 0.5221 - val_loss: 0.6908 - val_acc: 0.5339
Epoch 5/100
268/268 [==============================] - 1s 3ms/step - loss: 0.6861 - acc: 0.5459 - val_loss: 0.6581 - val_acc: 0.6134
Epoch 6/100
268/268 [==============================] - 1s 3ms/step - loss: 0.6310 - acc: 0.6541 - val_loss: 0.6010 - val_acc: 0.6996
Epoch 7/100
268/268 [==============================] - 1s 3ms/step - loss: 0.5771 - acc: 0.7204 - val_loss: 0.5525 - val_acc: 0.7486
Epoch 8/100
268/268 [==============================] - 1s 3ms/step - 

268/268 [==============================] - 1s 3ms/step - loss: 0.3317 - acc: 0.8830 - val_loss: 0.2468 - val_acc: 0.9290
Epoch 63/100
268/268 [==============================] - 1s 3ms/step - loss: 0.3338 - acc: 0.8836 - val_loss: 0.2666 - val_acc: 0.9295
Epoch 64/100
268/268 [==============================] - 1s 3ms/step - loss: 0.3329 - acc: 0.8864 - val_loss: 0.2352 - val_acc: 0.9327
Epoch 65/100
268/268 [==============================] - 1s 3ms/step - loss: 0.3299 - acc: 0.8830 - val_loss: 0.2458 - val_acc: 0.9311
Epoch 66/100
268/268 [==============================] - 1s 3ms/step - loss: 0.3251 - acc: 0.8869 - val_loss: 0.2525 - val_acc: 0.9295
Epoch 67/100
268/268 [==============================] - 1s 3ms/step - loss: 0.3385 - acc: 0.8801 - val_loss: 0.2557 - val_acc: 0.9311
Epoch 68/100
268/268 [==============================] - 1s 3ms/step - loss: 0.3241 - acc: 0.8860 - val_loss: 0.2432 - val_acc: 0.9321
Epoch 69/100
268/268 [==============================] - 1s 3ms/step - loss:

In [201]:
bnn.save_model(model, '../models/wishbone_naive', history=history)

INFO:tensorflow:Assets written to: ../models/wishbone_surface2/assets


'../models/wishbone_surface2'

In [203]:
signal_df_train.shape

(23762, 25)

In [204]:
surface_df_train

,110-CD3,115-CD45,139-CD45RA,142-CD19,144-CD11b,145-CD4,146-CD8,147-CD20,148-CD34,158-CD33,160-CD123,167-CD38,170-CD90,branch,trajectory
103281,-0.481132,248.784714,143.097122,54.863110,2.753687,13.147909,-1.370186,83.163406,4.977398,11.251880,13.962069,50.657806,1.466396,2,0.262880
388857,-0.974373,126.571587,9.464400,32.153496,-0.918507,-1.728501,-0.215477,12.470786,-0.274119,-0.340547,0.277593,11.110766,14.902977,1,0.058870
306517,-0.223668,144.316330,5.137239,16.677908,1.658641,-0.267913,-0.275669,28.161079,-0.848079,18.984800,1.359186,27.343784,-0.838904,1,0.058963
108228,-0.312490,176.851624,72.389610,38.293720,-0.849039,-0.913960,-0.269339,18.042175,-0.422611,10.066899,38.154644,36.512745,7.423636,2,0.263358
166493,0.801312,239.489166,80.141556,33.297813,11.679992,2.055430,0.395223,62.516212,8.888879,5.707616,6.729420,17.707584,5.695710,1,0.053682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325423,-0.144069,147.665237,55.637787,40.626232,0.942537,-0.595052,-4.129198,35.371353,-0.093601,8.387814,19.255114,81.409691,1.647623,2,0.263485
140523,-1.651225,273.520966,211.142822,60.851681,4.012023,-0.689578,0.725354,38.287445,4.667103,13.687876,-0.515647,78.021027,0.173368,2,0.257534
128201,-0.409057,73.494614,32.576759,22.963024,-0.821542,2.512493,14.791347,29.236551,2.743870,3.239280,-0.773837,40.983822,1.686235,2,0.265642
468129,-0.056041,95.279770,56.255215,15.047943,4.218547,3.787249,-0.641857,39.146057,-0.284752,1.224908,-0.264594,19.421221,3.261183,1,0.058928
